#Aplicaciones: detección de tópicos y análisis de sentimiento

Dos aplicaciones clásicas de procesamiento de lenguaje natural son la detección de tópicos y el análisis de sentimiento.
En el primer caso (tópicos) usaremos técnicas no supervisadas de reducción de dimensiones y en el segundo caso técnicas no supervisadas (bag of words) y supervisadas (bayes ingenuo,SVM, Random Forest)
Primero importemos algunas librerías

In [2]:
import sklearn
import os, re
import nltk

##Importación y limpieza del corpus
Vamos a importar 12 libros antiguos para ver si se clasifican en temas

In [1]:
#from google.colab import drive
#drive.mount("/content/drive/")

In [3]:
os.chdir("/content/drive/My Drive/corpus")
documents = []
for f in os.listdir():
    if f[-4:] == '.txt':
        documents.append(f[:-4])

In [4]:
contents = []
for document in documents:
    with open(document+'.txt', 'r', encoding="UTF-8") as f:
        contents.append(f.read())
len(contents)

13

Procedemos a hacer una limpieza que incluye inicio y final de los libros, caracteres especiales y remoción de stopwords.

In [5]:
for i in range(0,len(contents)):
    inicio=contents[i].find("EBOOK")
    final=contents[i].find("END OF")
    contents[i]=contents[i][inicio:final]

In [6]:
from nltk.corpus import stopwords
nltk.download("stopwords")
misstop=stopwords.words("spanish")+["á","Y"]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


definimos una función para remover stopwords

In [7]:
def filtrado(texto):
    filtrados=[word for word in texto if word not in misstop]
    return(filtrados)    


Hacemos la limpieza necesaria de caracteres y stopwords

In [8]:
nltk.download("punkt")
filtradito=[]

for i in range(len(contents)):
    contents[i] = re.sub("\""," ",contents[i])
    contents[i] = re.sub ("\n|\t"," ",contents[i])
    contents[i]=contents[i].lower()
    breve=nltk.tokenize.word_tokenize(contents[i],language="spanish")
    tempfilt=filtrado(breve)
    filtradito.append(tempfilt)
    
unidito=[]
for element in filtradito:
    unidito.append(" ".join(element))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


##Detección de tópicos (topic detection)

En detección de tópicos usaremos dos técnicas de reducción de dimensiones, ambas conectadas con SVD (singular value decomposition).
Ellas son: NMF (*negative matrix factorization*) y LDA (*latent dirlchet analysis*)

In [9]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation


Vamos a comparar los resultados sin limpiar los datos y limpiándolos. 
Primero obtenemos, para NMF, una representación tipo TF-IDF de los textos.

In [10]:
#opcion 1 sin limpiar
vectores=TfidfVectorizer(max_df=0.9,min_df=2,max_features=1000)
tfs=vectores.fit_transform(contents)
nombres=vectores.get_feature_names()


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


en cambio, para LDA usamos una representación de simple conteo

In [11]:
paralda = CountVectorizer(max_df=0.9, min_df=2, max_features=1000)
tflda=paralda.fit_transform(contents)
nombreslda=paralda.get_feature_names()


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


ahora ejecutamos el NMF

In [12]:
nmf = NMF(n_components=8, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfs)


/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1425: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  FutureWarning,


La factorización de la matriz de términos a documentos no obtiene como tal el nombre de los tópicos, el cual debe ser deducido a partir de los términos más representativos o importantes en el tópico. Por eso haremos una función que los extraiga.

In [13]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

Y ahora sí los obtenemos

In [14]:
no_top_words = 20
display_topics(nmf, nombres, no_top_words)


Topic 0:
había dijo después día tenía mí te fué rey oh sólo joven virrey don tu decía voz eran habían mujer
Topic 1:
habia habian martin quando indios dia despues tenia fué tambien tupac comandante rebeldes amaru demas ménos provincia dias josé vieja
Topic 2:
don to juan is luis gonzalo in diego virrey doña lima entendimiento enemigos espaldas encima espada esos escudos encontrar escuderos
Topic 3:
salsa manteca caldo despues pimienta sal cacerola cocer sopa cocido azúcar pescado póngase aceite pone fig tambien illustration añade ternera
Topic 4:
ulises telémaco pretendientes penélope júpiter minerva te palacio ítaca dioses tu nave neptuno néstor compañeros fué laertes circe xi héroe
Topic 5:
jesús vosotros discípulos judíos ley mí cristo tú testimonio cosas pecado dijo 14 20 te 11 12 15 18 respondió
Topic 6:
fábula pág imagen véase nota asno lobo león etc tu te _la perro fué júpiter dijo autor sólo decía _el
Topic 7:
quijote sancho don dijo respondió fue merced vuestra panza caballero

Aqui obtenemos la pertenencia de cada documento a su tópico

In [15]:
doc_topic = nmf.transform(tfs)
for n in range(doc_topic.shape[0]):
    topic_most_pr = doc_topic[n].argmax()
    print("doc: {} topic: {}\n".format(n,topic_most_pr))
print(documents)

doc: 0 topic: 0

doc: 1 topic: 1

doc: 2 topic: 0

doc: 3 topic: 7

doc: 4 topic: 0

doc: 5 topic: 2

doc: 6 topic: 3

doc: 7 topic: 0

doc: 8 topic: 1

doc: 9 topic: 0

doc: 10 topic: 5

doc: 11 topic: 6

doc: 12 topic: 4

['milyuna_t1', 'relacionhistoricasucesosdetupacamaru', 'tradiciones_peruanas_ricardo_palma', 'elquijote', 'cuentos_allan_poe', 'juan_tenorio', 'libro_cocina', 'obras-escogidas_becquer', 'candido-de-voltaire', 'el-buscon', 'nuevo_testamento_valera', 'fabulas_samaniego', 'odisea']


/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1425: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  FutureWarning,


comparemos con LDA

In [16]:

lda = LatentDirichletAllocation(n_components=8, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tflda)
id_topico=lda.fit_transform(tflda)
display_topics(lda,nombreslda,no_top_words)


Topic 0:
ulises te dijo oh rey después fué tu palacio día había telémaco sancho don júpiter tú quijote tus penélope mí
Topic 1:
manteca salsa sal caldo despues pone pimienta sirve aceite cacerola pan azúcar cocer pescado sopa cocido añade etc harina encima
Topic 2:
don dijo quijote sancho había respondió te vuestra merced mí fue caballero después tenía señora día tu cosas sólo habían
Topic 3:
jesús fué habia indios había vosotros habian dijo mí josé rebeldes dia general lima ley tupac tropas tambien despues cosas
Topic 4:
ulises te telémaco tu júpiter pretendientes palacio oh después dijo rey penélope minerva fué había dioses compañeros nave ánimo corazón
Topic 5:
don juan to luis dijo te ulises in is quijote gonzalo había oh diego sancho telémaco mí después tu júpiter
Topic 6:
fábula imagen pág véase nota etc león lobo perro tu asno fué _la te instante autor decía dijo sólo _el
Topic 7:
don sancho dijo quijote respondió te merced había vuestra caballero martin habia quando fue señora 

Obtengamos la asignación de tópicos a documentos

In [17]:
doc_topic = lda.transform(tfs)
for n in range(doc_topic.shape[0]):
    topic_most_pr = doc_topic[n].argmax()
    print("doc: {} topic: {}\n".format(n,topic_most_pr))
print(documents)

doc: 0 topic: 4

doc: 1 topic: 3

doc: 2 topic: 3

doc: 3 topic: 2

doc: 4 topic: 2

doc: 5 topic: 5

doc: 6 topic: 1

doc: 7 topic: 2

doc: 8 topic: 7

doc: 9 topic: 2

doc: 10 topic: 3

doc: 11 topic: 6

doc: 12 topic: 4

['milyuna_t1', 'relacionhistoricasucesosdetupacamaru', 'tradiciones_peruanas_ricardo_palma', 'elquijote', 'cuentos_allan_poe', 'juan_tenorio', 'libro_cocina', 'obras-escogidas_becquer', 'candido-de-voltaire', 'el-buscon', 'nuevo_testamento_valera', 'fabulas_samaniego', 'odisea']


Y ahora comparemos con bases limpias

In [18]:
tfslimpio=vectores.fit_transform(unidito)
nombreslimpio=vectores.get_feature_names()
nmflimpio = NMF(n_components=8, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfslimpio)
display_topics(nmflimpio, nombreslimpio, no_top_words)

Topic 0:
dijo después día rey fué oh sólo joven decía voz qué mujer virrey dije mientras don parecía podía cosas no
Topic 1:
habia habian martin quando indios dia despues tenia fué tambien tupac comandante rebeldes amaru demas ménos provincia dias josé vieja
Topic 2:
don to juan is luis gonzalo in virrey diego do doña lima perú en enamorado empezó emperador encantado encima encontrar
Topic 3:
salsa manteca caldo despues pimienta sal cacerola cocer sopa cocido azúcar pescado póngase aceite pone fig tambien illustration añade ternera
Topic 4:
jesús discípulos judíos ley cristo dijo cosas testimonio pecado 21 14 20 13 12 11 respondió 15 18 10 capítulo
Topic 5:
ulises telémaco pretendientes penélope júpiter minerva palacio ítaca dioses nave neptuno néstor compañeros fué laertes circe xi héroe ponto patria
Topic 6:
quijote sancho don dijo respondió merced panza caballero escudero señora mesmo dio vuesa vio duquesa cura andante ejercicio encontrar encima
Topic 7:
fábula pág véase imagen nota

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1425: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  FutureWarning,


In [19]:
tfldalimpio=paralda.fit_transform(unidito)
nombresldalimpio=paralda.get_feature_names()
ldalimpio = LatentDirichletAllocation(n_components=8, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tfldalimpio)
id_topico2=lda.fit_transform(tfldalimpio)
display_topics(ldalimpio,nombresldalimpio,no_top_words)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Topic 0:
don dijo ulises quijote sancho telémaco júpiter respondió después merced rey día pretendientes palacio sólo fué caballero algún cosas decía
Topic 1:
dijo después rey oh día joven fué sólo qué imagen voz mientras mujer fábula corazón aún no días por fondo
Topic 2:
manteca salsa sal caldo despues pone pimienta sirve aceite cacerola pan azúcar pescado cocer sopa añade cocido harina etc ponen
Topic 3:
jesús habia fué habian indios dijo dia josé despues rebeldes ley general tambien tropas tupac provincia cosas comandante amaru respondió
Topic 4:
sancho don quijote dijo respondió merced cosas caballero señora después oh rey día no panza cura decía sólo días mujer
Topic 5:
ulises telémaco pretendientes júpiter palacio don penélope fué minerva después dioses compañeros dijo oh nave juan ánimo iv héroe patria
Topic 6:
dijo después jesús fué manteca día don ulises cosas oh salsa caldo qué no telémaco quijote respondió sal pan sólo
Topic 7:
don quijote sancho dijo respondió merced caball

In [20]:
doc_topic = ldalimpio.transform(tfs)
for n in range(doc_topic.shape[0]):
    topic_most_pr = doc_topic[n].argmax()
    print("doc: {} topic: {}\n".format(n,topic_most_pr))
print(documents)

doc: 0 topic: 1

doc: 1 topic: 7

doc: 2 topic: 1

doc: 3 topic: 1

doc: 4 topic: 1

doc: 5 topic: 7

doc: 6 topic: 1

doc: 7 topic: 1

doc: 8 topic: 1

doc: 9 topic: 7

doc: 10 topic: 1

doc: 11 topic: 1

doc: 12 topic: 1

['milyuna_t1', 'relacionhistoricasucesosdetupacamaru', 'tradiciones_peruanas_ricardo_palma', 'elquijote', 'cuentos_allan_poe', 'juan_tenorio', 'libro_cocina', 'obras-escogidas_becquer', 'candido-de-voltaire', 'el-buscon', 'nuevo_testamento_valera', 'fabulas_samaniego', 'odisea']


In [21]:
doc_topic = nmflimpio.transform(tfs)
for n in range(doc_topic.shape[0]):
    topic_most_pr = doc_topic[n].argmax()
    print("doc: {} topic: {}\n".format(n,topic_most_pr))
print(documents)

doc: 0 topic: 0

doc: 1 topic: 3

doc: 2 topic: 0

doc: 3 topic: 7

doc: 4 topic: 0

doc: 5 topic: 0

doc: 6 topic: 0

doc: 7 topic: 0

doc: 8 topic: 0

doc: 9 topic: 0

doc: 10 topic: 0

doc: 11 topic: 0

doc: 12 topic: 0

['milyuna_t1', 'relacionhistoricasucesosdetupacamaru', 'tradiciones_peruanas_ricardo_palma', 'elquijote', 'cuentos_allan_poe', 'juan_tenorio', 'libro_cocina', 'obras-escogidas_becquer', 'candido-de-voltaire', 'el-buscon', 'nuevo_testamento_valera', 'fabulas_samaniego', 'odisea']


/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1425: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  FutureWarning,


##Análisis de sentimiento (Sentiment analysis)

En análisis de sentimiento es fundamental distinguir entre aproximaciones supervisadas y no supervisadas

### No supervisado: bag of words

Una aproximación simple pero poderosa es tener un buen diccionario de términos con su supuesta polaridad. Eso es lo que subiremos en el siguiente código

In [22]:
sentimientos={}
         
for line in open("/content/drive/My Drive/corpus/sentiment/13_CLS_Final.txt"):
    palabra, valor=line.split("\t")
    sentimientos[palabra]=int(valor)

sentimientos["bonito"]

1

Si simplemente sumamos la polaridad de los términos, obtenemos un valor de sentimiento

In [23]:
creada="Todos mis amigos son buenos. Me dan lo mejor"
tokenes=nltk.tokenize.word_tokenize(creada,language="spanish")
##es una lista. Debo volverlo texto de NLTK"
oracionpositiva=nltk.Text(tokenes)
sum(sentimientos.get(palabra,0) for palabra in oracionpositiva)   


3

In [24]:
creada2="todos político es corrupto"
tokenes2=nltk.tokenize.word_tokenize(creada2,language="spanish")
##es una lista. Debo volverlo texto de NLTK"
oracionnegativa=nltk.Text(tokenes2)
sum(sentimientos.get(palabra,0) for palabra in oracionnegativa)   

-1

In [25]:
type(tokenes2)

list

Podemos hacerlo inclusive para un libro completo

In [26]:
sum(sentimientos.get(palabra,0) for palabra in filtradito[2])   


1237

Y ahora, valoremos todos nuestros tuits de manera no supervisada

In [27]:
posbow=[]
negbow=[]

#lleno listas positivas y negativas con mis bases anotadas"
with open("/content/drive/My Drive/corpus/sentiment/tass14pos.txt", encoding="latin-1") as f:
    for line in f:
        posbow.append(line)

with open("/content/drive/My Drive/corpus/sentiment/tass14neg.txt", encoding="latin-1") as f:
    for line in f:
        negbow.append(line)



Hagamos las predicciones en un bucle

In [28]:
import numpy as np
predic_pos=np.zeros(len(posbow))
i=0
for elemento in posbow:
  tokenestemp=nltk.tokenize.word_tokenize(elemento,language="spanish")
  predic_pos[i]=sum(sentimientos.get(palabra,0) for palabra in tokenestemp)
  i=i+1 

predic_neg=np.zeros(len(negbow))
i=0
for elemento in negbow:
  tokenestemp=nltk.tokenize.word_tokenize(elemento,language="spanish")
  predic_neg[i]=sum(sentimientos.get(palabra,0) for palabra in tokenestemp)
  i=i+1   




Y ahora clasifiquemos en positivos y negativos

In [29]:
import pandas as pd
pospanda=pd.DataFrame(predic_pos, columns=["sentiment"])
pospanda["prediccion"]=np.where(pospanda["sentiment"]>0,1,0)
pospanda["real"]=1

negpanda=pd.DataFrame(predic_neg,columns=["sentiment"] )
negpanda["prediccion"]=np.where(negpanda["sentiment"]>0,1,0)
negpanda["real"]=0

predic_tot=pd.concat([pospanda,negpanda])
display(predic_tot)


,sentiment,prediccion,real
0,0.0,0,1
1,2.0,1,1
2,1.0,1,1
3,3.0,1,1
4,1.0,1,1
...,...,...,...
1586,-2.0,0,0
1587,0.0,0,0
1588,-1.0,0,0
1589,1.0,1,0


In [30]:
from sklearn.metrics import precision_recall_curve, roc_curve, balanced_accuracy_score, f1_score, precision_score,recall_score, roc_auc_score
sklearn.metrics.precision_score(predic_tot["real"],predic_tot["prediccion"])

0.7729930887825625

In [31]:
bowr=sklearn.metrics.recall_score(predic_tot["real"],predic_tot["prediccion"])
bowf=sklearn.metrics.f1_score(predic_tot["real"],predic_tot["prediccion"])
bowauc=sklearn.metrics.roc_auc_score(predic_tot["real"],predic_tot["prediccion"])
print(bowr,bowf,bowauc)

0.5367294204503507 0.6335511982570807 0.6341723783584249


### Sentimiento supervisado: Bayes ingenuo

si tenemos casos positivos y negativos, podemos representar el texto simplemente como su presencia/ausencia de términos, y usar dichos términos como predictores de la polaridad. Una aproximación que supone (de manera fuerte) independencia entre las plaabras es Naive Bayes o Bayes ingenuo.
Empecemos por leer las palabras y hacer la representación:

In [32]:
def formato(tuit):
    return{palabra:True for palabra in nltk.tokenize.word_tokenize(tuit)}

formato("Estamos haciendo una prueba")


{'Estamos': True, 'haciendo': True, 'prueba': True, 'una': True}

Luego leemos los ejemplos positivos y negativos

In [33]:
pos=[]
neg=[]

#lleno listas positivas y negativas con mis bases anotadas"
with open("/content/drive/My Drive/corpus/sentiment/tass14pos.txt", encoding="latin-1") as f:
    for line in f:
        pos.append([formato(line),"pos"])

with open("/content/drive/My Drive/corpus/sentiment/tass14neg.txt", encoding="latin-1") as f:
    for line in f:
        neg.append([formato(line),"neg"])

pos[1]


[{'!': True,
  'Conozco': True,
  'Ja': True,
  'a': True,
  'adicto': True,
  'al': True,
  'algo': True,
  'alguien': True,
  'd': True,
  'drama': True,
  'es': True,
  'ja': True,
  'q': True,
  'suena': True,
  'te': True},
 'pos']

In [34]:
len(pos)


2709

In [35]:
len(neg)

1591

Y creamos bases de entrenamiento y validación

In [36]:
entrenamiento=pos[:1280]+neg[:1280]
validacion=pos[1280:]+neg[1280:]


Ahora aplicamos el modelo

In [37]:
"traer el modelo de machine learning a aplicar"
from nltk.classify import NaiveBayesClassifier
"creo el modelo en la base de datos de entrenamiento"
model=NaiveBayesClassifier.train(entrenamiento)


Y ya podemos probarlo!

In [38]:
print(model.classify(formato("hicieron lo que pudieron a final de cuentas")))


neg


In [39]:
print(model.classify(formato("Todos los políticos son unos corruptos")))


neg


In [40]:
print(model.classify(formato("Ninguno de los políticos es corrupto")))


neg


In [41]:
print(model.classify(formato("Pero si todos los políticos son unos angeles maravillosos")))


neg


In [42]:
print(model.classify(formato("todos los profesores son unos angeles maravillosos")))


neg


In [43]:
print(model.classify(formato("estoy a favor de la pena de muerte")))


pos


Una prueba más formal, con indicadores

In [44]:
from nltk.classify.util import accuracy

accuracy(model,validacion)


0.5839080459770115

Separo la representación de lo que quiero predecir

In [79]:
representa=[]
for item in validacion:
  representa.append(item[0])

realnb=[]
for item in validacion:
  realnb.append(item[1])



Realizo las predicciones

In [80]:
naive=[]
i=0
for elemento in representa:
  naive.append(model.classify(elemento))


Calculo métricas

In [87]:
nbprec=sklearn.metrics.precision_score(realnb,naive, pos_label="pos")
nbrecall=sklearn.metrics.recall_score(realnb,naive, pos_label="pos")
nbfscore=sklearn.metrics.f1_score(realnb,naive, pos_label="pos")
#nbauc=sklearn.metrics.roc_auc_score(realnb,naive)
print(nbprec,nbrecall,nbfscore)

0.8886438809261301 0.5640307907627712 0.6900684931506849


###Sentimiento supervisado: SVM

La suposición de independencia es bastante fuerte en Naive Bayes. La representación por simple presencia/ausencia es también simple.
Vamos a sofisticar la aproximación tanto en representación (TF) como en algoritmo de aprendizaje (SVM). Primero leemos los ejemplos:

In [88]:
positivo=[]
with open("/content/drive/My Drive/corpus/sentiment/tass14pos.txt", encoding="latin-1") as f:
    for line in f:
        positivo.append(line)
len(positivo)

2709

In [89]:
negativo=[]
with open("/content/drive/My Drive/corpus/sentiment/tass14neg.txt", encoding="latin-1") as f:
    for line in f:
        negativo.append(line)
total=positivo + negativo
len(negativo)

1591

Luego hacemos la representación.

In [90]:
conteos= CountVectorizer(max_df=0.9, min_df=2, max_features=500)
total2=conteos.fit_transform(total)
matrizt=total2.todense()
matlistt=matrizt.tolist()
matlistt[10]


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


Creamos la variable de respuesta

In [91]:
respuesta=[1]*len(positivo)+ [0]*len(negativo)

respuesta[3800]


0

Y dividimos en entrenamiento y validación

In [92]:
from sklearn.model_selection import train_test_split
nlp_tr, nlp_val, sentiment_tr, sentimen_val = train_test_split(matlistt, respuesta, test_size=0.05, random_state=42, stratify=respuesta)


In [93]:
len(nlp_tr)


4085

ahora creamos el modelo y lo ejecutamos sobre nuestros datos

In [95]:
from sklearn import svm
svmsent=svm.SVC(kernel="rbf",gamma="scale", probability=True, class_weight="balanced")
svmsent.fit(nlp_tr,sentiment_tr)


SVC(class_weight='balanced', probability=True)

Creamos predicciones

In [96]:
predicciones=svmsent.predict(nlp_val)
predicciones.shape


(215,)

Y lo medimos con diferentes métricas

In [97]:
from sklearn.metrics import precision_recall_curve, roc_curve, balanced_accuracy_score, f1_score, precision_score,recall_score, roc_auc_score
sklearn.metrics.precision_score(sentimen_val,predicciones)

0.7835820895522388

In [98]:
sklearn.metrics.recall_score(sentimen_val,predicciones)


0.7777777777777778

In [99]:
sklearn.metrics.f1_score(sentimen_val,predicciones)



0.7806691449814127

In [100]:
sklearn.metrics.roc_auc_score(sentimen_val,predicciones)

0.7076388888888887

In [101]:
sklearn.metrics.precision_score(sentimen_val,predicciones, pos_label=0)


0.6296296296296297

In [102]:
sklearn.metrics.recall_score(sentimen_val,predicciones, pos_label=0)


0.6375

In [103]:
sklearn.metrics.f1_score(sentimen_val,predicciones, pos_label=0)

0.6335403726708074

Se generan dos conclusiones: Si bien mejorar la aproximación de representación y el algoritmo aumentan las métricas de desempeño, eso puede venir a costa de esfuerzos computacionales, por lo que debe considerarse el volumen de datos a la hora de elegir una aproximación.

Haremos finalmente un Random Forest

In [109]:
from sklearn.ensemble import RandomForestClassifier
sentimentrf=RandomForestClassifier(n_estimators=300,random_state=67, class_weight="balanced")
sentimentrf.fit(nlp_tr,sentiment_tr)

RandomForestClassifier(class_weight='balanced', n_estimators=300,
                       random_state=67)

In [110]:
prediccionesrf=sentimentrf.predict(nlp_val)

In [111]:
rfprec=sklearn.metrics.precision_score(sentimen_val,prediccionesrf)
rfrecall=sklearn.metrics.recall_score(sentimen_val,prediccionesrf)
rffscore=sklearn.metrics.f1_score(sentimen_val,prediccionesrf)
rfauc=sklearn.metrics.roc_auc_score(sentimen_val,prediccionesrf)
print(rfprec,rfrecall,rffscore,rfauc)

0.7423312883435583 0.8962962962962963 0.8120805369127516 0.6856481481481481
